This notebook is joint work by Harmony Yap and Nanxi Zhang

In [ ]:
import os
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

from bs4 import BeautifulSoup
import os
import requests
import pandas as pd
import numpy as np

In [108]:
PATH = "Data_Files/ova"

In [132]:
DATA_PATH = 'Data_Files/epl-training.csv'
data = pd.read_csv(DATA_PATH,index_col=0)
data = data.dropna(axis=1, how='all')
data.head()
data.index= pd.to_datetime(data.index)
data = data.sort_index()

In [110]:
index = [i for i in range(2008,2022)]
columns = data['HomeTeam'].unique()
ova_df = pd.DataFrame(index=index, columns=columns)

In [111]:
ova_df['Leeds'] = np.nan
ova_df['Derby'] = np.nan
ova_df['Charlton'] = np.nan
ova_df['Watford'] = np.nan

In [199]:
def team_name_mapper(name):
    team_name_map = {
        'Birmingham City': 'Birmingham',
        'Blackburn Rovers': 'Blackburn',
        'Bolton Wanderers': 'Bolton',
        'Brighton & Hove Albion': 'Brighton',
        'Cardiff City': 'Cardiff',
        'Charlton Athletic': 'Charlton',
        'Derby County': 'Derby',
        'Huddersfield Town': 'Huddersfield',
        'Hull City': 'Hull',
        'Leeds United':'Leeds',
        'Leicester City': 'Leicester',
        'Manchester City': 'Man City',
        'Manchester United': 'Man United',
        'Newcastle United': 'Newcastle',
        'Norwich City': 'Norwich',
        'Queens Park Rangers': 'QPR',
        'Stoke City': 'Stoke',
        'Swansea City': 'Swansea',
        'Tottenham Hotspur': 'Tottenham',
        'West Bromwich Albion': 'West Brom',
        'West Ham United': 'West Ham',
        'Wigan Athletic': 'Wigan',
        'Wolverhampton Wanderers': 'Wolves',
    }
    if name in team_name_map:
        return team_name_map[name]
    else:
        return name

In [112]:
def fifa_ova_scrapper(path,start,end,all_df):
    if not os.path.exists(path):
        os.makedirs(path)
    browser = webdriver.Chrome(ChromeDriverManager().install())
    browser.get("https://sofifa.com/teams?type=all&lg%5B%5D=13")
    
    for year in range(start,end+1):
        browser.find_element_by_xpath("//a[contains(@class, 'bp3-button bp3-minimal text dropdown-toggle')]").click()
        time.sleep(1)
        browser.find_element_by_xpath("//a[contains(text(), 'FIFA " + str(format((year % 2000 + 1), '02d')) + "')]").click()
        time.sleep(1)
        teams = [team.text for team in browser.find_elements_by_xpath("//tbody/tr/td[2]/a/div")[::2]]
        ovas = [ova.text for ova in browser.find_elements_by_xpath("//tbody/tr/td[3]/span")]
        df = pd.DataFrame.from_records(zip(teams,ovas), columns=["team", "ova"])
        df['team']=df['team'].apply(team_name_mapper)
        for i in df.index:
            ova_df[df['team'][i]][year+1]=df['ova'][i]
        df.set_index('team', inplace=True)
        df.to_csv("{}/{}.csv".format(path, year+1))
        print("Collected data for", year)

In [114]:
fifa_ova_scrapper(PATH,2006, 2020,ova_df)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/nanxizhang/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


<ipython-input-112-aecc58545cbb>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ova_df[df['team'][i]][year+1]=df['ova'][i]


Collected data for 2006
Collected data for 2007
Collected data for 2008
Collected data for 2009
Collected data for 2010
Collected data for 2011
Collected data for 2012
Collected data for 2013
Collected data for 2014
Collected data for 2015
Collected data for 2016
Collected data for 2017
Collected data for 2018
Collected data for 2019
Collected data for 2020


In [171]:
ova_df_test = ova_df.copy()
ova_df_test=ova_df_test.fillna(0)

In [245]:
def clear_history_based(df):
    
    fix= dict.fromkeys(df.columns,0) 
    for team in df.columns:
        if 0 not in df[team]:
            next
        for i in range(2008,2022):
            if df[team].loc[i]==0:
                df[team].loc[i]=fix[team]
            else:
                fix[team]=df[team].loc[i]

In [246]:
def clear_future_based(df):
    fix= dict.fromkeys(df.columns,0) 
    for team in df.columns:
        if 0 not in df[team]:
            next
        for i in range(2021,2007,-1):
            if df[team].loc[i]==0:
                df[team].loc[i]=fix[team]
            else:
                fix[team]=df[team].loc[i]

In [174]:
clear_history_based(ova_df_test)
clear_future_based(ova_df_test)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [189]:
ova_df_test=ova_df_test.drop('Charlton',axis = 1)

In [190]:
ova_df_test

,Arsenal,Bolton,Everton,Hull,Middlesbrough,Sunderland,West Ham,Aston Villa,Chelsea,Man United,...,Crystal Palace,Cardiff,Leicester,Bournemouth,Watford,Brighton,Huddersfield,Sheffield United,Leeds,Derby
2008,80,75,76,71,75,71,76,75,83,83,...,72,72,72,73,75.0,75,74,75,75.0,69.0
2009,81,74,78,71,74,75,76,78,84,84,...,72,72,72,73,75.0,75,74,75,75.0,69.0
2010,81,75,79,73,74,75,76,79,83,82,...,72,72,72,73,75.0,75,74,75,75.0,69.0
2011,81,75,78,73,74,75,75,78,83,82,...,72,72,72,73,75.0,75,74,75,75.0,69.0
2012,81,74,77,73,74,76,75,76,83,83,...,72,72,72,73,75.0,75,74,75,75.0,69.0
2013,80,74,76,73,74,75,74,75,81,82,...,72,72,72,73,75.0,75,74,75,75.0,69.0
2014,80,74,78,72,74,74,74,74,82,82,...,72,72,72,73,75.0,75,74,75,75.0,69.0
2015,80,74,78,73,74,74,75,75,83,81,...,73,72,72,73,75.0,75,74,75,75.0,69.0
2016,82,74,78,73,74,75,77,75,83,81,...,76,72,76,73,75.0,75,74,75,75.0,69.0
2017,83,74,80,75,75,76,78,75,83,83,...,77,72,78,74,77.0,75,74,75,75.0,69.0


In [191]:
ova_df_test.to_csv("{}/all_team_ova.csv".format(PATH))

In [197]:
PATH_PLAYER = "Data_Files/player_stats"

In [194]:
def scrap_data(url,players=False):
    html = requests.get(url)
    soup = BeautifulSoup(html.content,'html.parser')
    columns = [i['data-tooltip'] for i in soup.find(class_='persist-header').find_all(class_='tooltip')]
    
    if players:
        all_data = soup.find(class_='list').find_all('tr')
        data = [[j.string for j in i.find_all(class_='col')] for i in all_data]
        columns.remove('Team & Contract')
        columns.remove('Hits')
    else:
        table = soup.find_all('tr')[1:]
        data = [[i.string for i in j.find_all(class_='col')] for j in table]
    result = pd.DataFrame(data=data,columns=columns)
    if players:
        teams = [i.find_all('a',href=True)[-1].text for i in all_data]
        result['Team'] = pd.Series(teams)
    else:
        teams = [i.find_all('a',href=True)[0].text for i in table]
        result['Team'] = pd.Series(teams)
    return result

In [195]:
def loop_offsets(url,max_offset,year,path):
    df = scrap_data(url+'&offset='+str(0),players=True)
    for i in range(1,max_offset//60+1):
        
        data = scrap_data(url+'&offset='+str(i*60),players=True)
        df = df.append(data) 
    df['Team']=df['Team'].apply(team_name_mapper)
    if not os.path.exists(path):
        os.makedirs(path)
    df.to_csv("{}/{}.csv".format(path, str(year)))
    return df

In [200]:
d2021=loop_offsets('https://sofifa.com/players?type=all&lg%5B0%5D=13',600,2021,PATH_PLAYER)
d2020=loop_offsets('https://sofifa.com/players?type=all&lg%5B0%5D=13&r=200061&set=true',600,2020,PATH_PLAYER)
d2019=loop_offsets('https://sofifa.com/players?type=all&lg%5B0%5D=13&r=190075&set=true',600,2019,PATH_PLAYER)
d2018=loop_offsets('https://sofifa.com/players?type=all&lg%5B0%5D=13&r=180084&set=true',600,2018,PATH_PLAYER)
d2017=loop_offsets('https://sofifa.com/players?type=all&lg%5B0%5D=13&r=170099&set=true',600,2017,PATH_PLAYER)
d2016=loop_offsets('https://sofifa.com/players?type=all&lg%5B0%5D=13&r=160058&set=true',600,2016,PATH_PLAYER)
d2015=loop_offsets('https://sofifa.com/players?type=all&lg%5B0%5D=13&r=150059&set=true',600,2015,PATH_PLAYER)
d2014=loop_offsets('https://sofifa.com/players?type=all&lg%5B0%5D=13&r=140052&set=true',600,2014,PATH_PLAYER)
d2013=loop_offsets('https://sofifa.com/players?type=all&lg%5B0%5D=13&r=130034&set=true',600,2013,PATH_PLAYER)
d2012=loop_offsets('https://sofifa.com/players?type=all&lg%5B0%5D=13&r=120002&set=true',600,2012,PATH_PLAYER)
d2011=loop_offsets('https://sofifa.com/players?type=all&lg%5B0%5D=13&r=110002&set=true',600,2011,PATH_PLAYER)
d2010=loop_offsets('https://sofifa.com/players?type=all&lg%5B0%5D=13&r=100002&set=true',600,2010,PATH_PLAYER)
d2009=loop_offsets('https://sofifa.com/players?type=all&lg%5B0%5D=13&r=090002&set=true',600,2009,PATH_PLAYER)
d2008=loop_offsets('https://sofifa.com/players?type=all&lg%5B0%5D=13&r=080002&set=true',480,2008,PATH_PLAYER)

In [247]:
index_ps = [i for i in range(2008,2022)]
columns_ps = ova_df_test.columns
ova_ps = pd.DataFrame(index=index_ps, columns=columns_ps)

In [249]:
def yearly_player_data(df,year):
    year_data = pd.read_csv('Data_Files/player_stats/'+str(year)+'.csv',index_col=0)
    year_data['Total Stats'] = pd.to_numeric(year_data['Total Stats'], errors='coerce')
    teams = year_data['Team'].unique()
    for team in teams:
        df[team].loc[year]=year_data[year_data['Team']=='Man United']['Total Stats'].mean()

In [250]:
def get_overall_player_data(fromyear, toyear,df):
    for i in range(fromyear,toyear+1):
        yearly_player_data(df,i)

In [251]:
get_overall_player_data(2008,2021,ova_ps)

In [267]:
ova_ps_c = ova_ps.copy()
ova_ps_c=ova_ps_c.fillna(0)
clear_history_based(ova_ps_c)
clear_future_based(ova_ps_c)

In [270]:
ova_ps_c

,Arsenal,Bolton,Everton,Hull,Middlesbrough,Sunderland,West Ham,Aston Villa,Chelsea,Man United,...,Crystal Palace,Cardiff,Leicester,Bournemouth,Watford,Brighton,Huddersfield,Sheffield United,Leeds,Derby
2008,1523.880000,1523.880000,1523.880000,1434.057143,1523.880000,1523.880000,1523.880000,1523.880000,1523.880000,1523.880000,...,1773.212121,1773.212121,1762.161290,1804.515152,1804.515152,1877.939394,1877.939394,1874.666667,1890.363636,1523.88
2009,1434.057143,1434.057143,1434.057143,1434.057143,1434.057143,1434.057143,1434.057143,1434.057143,1434.057143,1434.057143,...,1773.212121,1773.212121,1762.161290,1804.515152,1804.515152,1877.939394,1877.939394,1874.666667,1890.363636,1523.88
2010,1436.275000,1436.275000,1436.275000,1436.275000,1434.057143,1436.275000,1436.275000,1436.275000,1436.275000,1436.275000,...,1773.212121,1773.212121,1762.161290,1804.515152,1804.515152,1877.939394,1877.939394,1874.666667,1890.363636,1523.88
2011,1851.742857,1851.742857,1851.742857,1436.275000,1434.057143,1851.742857,1851.742857,1851.742857,1851.742857,1851.742857,...,1773.212121,1773.212121,1762.161290,1804.515152,1804.515152,1877.939394,1877.939394,1874.666667,1890.363636,1523.88
2012,1787.030303,1787.030303,1787.030303,1436.275000,1434.057143,1787.030303,1851.742857,1787.030303,1787.030303,1787.030303,...,1773.212121,1773.212121,1762.161290,1804.515152,1804.515152,1877.939394,1877.939394,1874.666667,1890.363636,1523.88
2013,1764.515152,1787.030303,1764.515152,1436.275000,1434.057143,1764.515152,1764.515152,1764.515152,1764.515152,1764.515152,...,1773.212121,1773.212121,1762.161290,1804.515152,1804.515152,1877.939394,1877.939394,1874.666667,1890.363636,1523.88
2014,1773.212121,1787.030303,1773.212121,1773.212121,1434.057143,1773.212121,1773.212121,1773.212121,1773.212121,1773.212121,...,1773.212121,1773.212121,1762.161290,1804.515152,1804.515152,1877.939394,1877.939394,1874.666667,1890.363636,1523.88
2015,1762.161290,1787.030303,1762.161290,1762.161290,1434.057143,1762.161290,1762.161290,1762.161290,1762.161290,1762.161290,...,1762.161290,1773.212121,1762.161290,1804.515152,1804.515152,1877.939394,1877.939394,1874.666667,1890.363636,1523.88
2016,1804.515152,1787.030303,1804.515152,1762.161290,1434.057143,1804.515152,1804.515152,1804.515152,1804.515152,1804.515152,...,1804.515152,1773.212121,1804.515152,1804.515152,1804.515152,1877.939394,1877.939394,1874.666667,1890.363636,1523.88
2017,1841.636364,1787.030303,1841.636364,1841.636364,1841.636364,1841.636364,1841.636364,1804.515152,1841.636364,1841.636364,...,1841.636364,1773.212121,1841.636364,1841.636364,1841.636364,1877.939394,1877.939394,1874.666667,1890.363636,1523.88


In [274]:
ova_ps_c.to_csv("{}/all_players_stats.csv".format(PATH_PLAYER))